In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, mean_squared_error, mean_absolute_error, r2_score
import scipy.stats as stats
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression


In [ ]:
df = pd.read_csv("SupplyChain_preprocessed.csv")

<ipython-input-67-ca8d0b4c8251>:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("SupplyChain_preprocessed.csv")


In [ ]:
df['order_year'] = pd.DatetimeIndex(df['Order Date']).year

In [ ]:
df.drop(["Product Category Id", "Product Card Id", "Order Item Id", "Order Item Cardprod Id", "Order Id", "Order Customer Id", "Department Id", "Customer Id", "Category Id", "Type", "Product Status", "Product Image", "Order Date", "Shipping Date"],axis=1,inplace=True)

In [ ]:
sales_2015_2018 = df[df['order_year'].isin([2015, 2016, 2017, 2018])]

In [ ]:
sales_2015_2018.shape

(180519, 35)

In [ ]:
sales_2015_2018 = sales_2015_2018.apply(lambda x: x.astype('category').cat.codes if x.dtype == 'object' else x)

print(sales_2015_2018)

        Days for shipping (real)  Days for shipment (scheduled)  \
0                              3                              4   
1                              5                              4   
2                              4                              4   
3                              3                              4   
4                              2                              4   
...                          ...                            ...   
180514                         4                              4   
180515                         3                              2   
180516                         5                              4   
180517                         3                              4   
180518                         4                              4   

        Benefit per order  Sales per customer  Delivery Status  \
0               91.250000          314.640015                0   
1             -249.089996          311.359985                1 

In [ ]:
X_train =  sales_2015_2018[sales_2015_2018['order_year'].isin([2015, 2016, 2017])].drop(columns=['Sales', 'order_year', 'Late_delivery_risk'])  # Drop 'sales' and 'year' columns
y_train = sales_2015_2018[sales_2015_2018['order_year'].isin([2015, 2016, 2017])]['Sales']
X_test = sales_2015_2018[sales_2015_2018['order_year'] == 2018].drop(columns=['Sales', 'order_year', 'Late_delivery_risk'])  # Drop 'sales' and 'year' columns
y_test = sales_2015_2018[sales_2015_2018['order_year'] == 2018]['Sales']

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(X_train, y_train)

LinearRegression()

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)
print("R-squared:", r2)

Mean Squared Error: 8.533437721076402e-06
Mean Absolute Error: 0.00111492952322307
R-squared: 0.9999999995522961


In [ ]:
comparison_df = pd.DataFrame({
    'Actual Sales': y_test,
    'Predicted Sales': y_pred,
    'Difference': y_test - y_pred
})

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(y_test.values, label='Actual Sales')
plt.plot(y_pred, label='Predicted Sales', linestyle='--')
plt.xlabel('Index')
plt.ylabel('Sales')
plt.title('Actual vs Predicted Sales for 2018')
plt.legend()
plt.show()

# Plotting the differences
plt.figure(figsize=(10, 6))
plt.plot(comparison_df['Difference'], label='Difference')
plt.xlabel('Index')
plt.ylabel('Difference')
plt.title('Difference between Actual and Predicted Sales for 2018')
plt.legend()
plt.show()

In [ ]:
print(y_test-y_pred)

0        -0.000350
1        -0.000238
2        -0.000215
3        -0.000096
4         0.000042
            ...   
176826    0.000017
176827    0.000011
176828    0.000184
176837   -0.000097
179218    0.000114
Name: Sales, Length: 2123, dtype: float64
